# Importing important libraries

In [1]:
import time
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import re

In [42]:
#connect to times of india webpage
sURL="https://timesofindia.indiatimes.com/"
driver=webdriver.Chrome("/Users/bhupendrakumar/Documents/Personal/chromedriver")
driver.get(sURL)

In [4]:
# Navigate to archives page
link=driver.find_element(By.LINK_TEXT,"Archives")
Archives_Address=link.get_attribute('href')
driver.get(Archives_Address)
time.sleep(3)

In [5]:
# Grab data for Feb 2018
links=driver.find_elements(By.LINK_TEXT,"February")

from datetime import date
d=date.today()
ActiveYear=int(d.strftime("%Y"))
YearToBesearched=2018
Link_Index=ActiveYear-YearToBesearched
driver.get(links[Link_Index].get_attribute('href'))

In [6]:
#Store links of days
Days_links=[]
for iDay in range(1,29):
    webaddress=driver.find_element(By.LINK_TEXT,str(iDay)).get_attribute('href')
    Days_links.append(webaddress)

In [7]:
#Store news links
News_links=[]
for iDay in Days_links:
    driver.get(iDay)
    time.sleep(2)
    #Set of news links
    News_Addresses=driver.find_elements_by_xpath("//span[@style='font-family:arial ;font-size:12;color: #006699']")
    
    #News links
    Text_Links=News_Addresses[0].text
    texts=Text_Links.split("\n")
    pattern=r"[a-zA-Z]+"
    for sText in texts:    
        if re.search(pattern,sText):
            webaddress=driver.find_element(By.LINK_TEXT,sText).get_attribute('href')
            News_links.append(webaddress)

## Defining and intializing lists

In [30]:
Headlines_list=[]
Authors_list=[]
Dates_list=[]
News_list=[]
Verticals_list=[]

In [46]:
for sNews_Link in News_links[5400:]:
    driver.get(sNews_Link)
    time.sleep(3)
    try:
        driver.find_element_by_xpath("//div[@class='error']")
    except NoSuchElementException:
        #Extracting Headlines
        try:
            headline=driver.find_element_by_xpath("//h1[@class='_23498']")
            headline=headline.text
        except ElementNotInteractableException:
            headline=driver.find_element_by_xpath("//h1[@class='heading1']")
            headline=headline.text
        except NoSuchElementException:
            try:
                headline=driver.find_element_by_xpath("//h1[@class='heading1']")
                headline=headline.text
            except NoSuchElementException:
                headline="-"
        Headlines_list.append(headline)

        #Extracting Authors and Dates
        try:
            Author_Date=driver.find_element_by_xpath("//div[@class='_3Mkg- byline']").text
            if "|" in Author_Date:
                Authors_list.append(Author_Date[:Author_Date.index(" | ")])
            else:
                Authors_list.append("-")
                
            if "Feb" not in Author_Date:
                Dates_list.append("-")
            else:
                Dates_list.append(Author_Date[Author_Date.index("Feb"):])
                
        except ElementNotInteractableException:
            Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
            Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
            Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Feb")])
            if "Feb" not in Author_Date:
                Dates_list.append("-")
            else:
                Dates_list.append(Author_Date[Author_Date.index("Feb"):])
        except NoSuchElementException:
            try:
                Author_Date=driver.find_element_by_xpath("//div[@class='as_byline']")
                Author_Date=Author_Date.text.split('\n')[0].replace("Created:","")
                Authors_list.append(Author_Date[Author_Date.index("-")+2:Author_Date.index("Feb")])
                if "Feb" not in Author_Date:
                    Dates_list.append("-")
                else:
                    Dates_list.append(Author_Date[Author_Date.index("Feb"):])
            except NoSuchElementException:
                Dates_list.append("-")
                Authors_list.append("-")
        #Extracting Description
        try:
            News=driver.find_element_by_xpath("//div[@class='ga-headlines']").text
        except ElementNotInteractableException:
            News=driver.find_element_by_xpath("//div[@class='Normal']").text
            News=News.replace("\n","")
        except NoSuchElementException:
            News=driver.find_element_by_xpath("//div[@class='Normal']").text
            News=News.replace("\n","")
        News_list.append(News)

        #Extracting Verticals
        try:
            Vertical=driver.find_elements_by_xpath("//span[@itemprop='name']")
            if len(Vertical)>=2:
                Vert=Vertical[1].text
            else:
                Vert="-"
        except ElementNotInteractableException:
            Vert="-"
        except NoSuchElementException:
            Vert="-"
        
        Verticals_list.append(Vert)


## Storing Data in CSV file

In [48]:
#Declaring the dataframe
Scrapped_data=pd.DataFrame()

#Initializing columns in the new data frame
Scrapped_data['Date']=Dates_list
Scrapped_data['Author']=Authors_list
Scrapped_data['Vertical']=Verticals_list
Scrapped_data['Headline']=Headlines_list
Scrapped_data['Description']=News_list

In [68]:
#Storing data in csv file
Scrapped_data.to_csv("/Users/bhupendrakumar/Documents/Personal/Internship/News Classification And Analysis/News Classification and Analysis Phase-1/ToI_data.csv")